In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Paths to the dataset
train_path = r"D:\Brain_Tumor_Detection\Training"
test_path = r"D:\Brain_Tumor_Detection\Testing"


In [3]:
# Transformations for the images
transforming_img = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# DataLoaders
train_loader = DataLoader(
    datasets.ImageFolder(train_path, transform=transforming_img),
    batch_size=64, shuffle=True
)

test_loader = DataLoader(
    datasets.ImageFolder(test_path, transform=transforming_img),
    batch_size=32, shuffle=True
)

# Extracting class names
classes = os.listdir(train_path)
print(f"The classes are: {classes}")


The classes are: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [4]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=4):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(12, 20, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(20, 32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.relu3 = nn.ReLU()

        self.fc = nn.Linear(32 * 75 * 75, num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = output.view(-1, 32 * 75 * 75)
        output = self.fc(output)
        return output

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConvNet(num_classes=len(classes)).to(device)


In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()


In [6]:
# Training loop
best_accuracy = 0.0
train_count = len(train_loader.dataset)
test_count = len(test_loader.dataset)

for epoch in range(20):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predictions = torch.max(outputs, 1)
        train_accuracy += (predictions == labels).sum().item()

    train_accuracy /= train_count
    train_loss /= train_count

    # Evaluate on test set
    model.eval()
    test_accuracy = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            test_accuracy += (predictions == labels).sum().item()

    test_accuracy /= test_count

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

    # Save the model if test accuracy improves
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), r"D:\Brain_Tumor_Detection\best_checkpoint.pth")
        best_accuracy = test_accuracy


Epoch 1, Train Loss: 5.5506, Train Accuracy: 0.6296, Test Accuracy: 0.3909
Epoch 2, Train Loss: 1.0847, Train Accuracy: 0.7923, Test Accuracy: 0.4898
Epoch 3, Train Loss: 0.9634, Train Accuracy: 0.8317, Test Accuracy: 0.5178
Epoch 4, Train Loss: 0.4943, Train Accuracy: 0.8948, Test Accuracy: 0.6548
Epoch 5, Train Loss: 0.4497, Train Accuracy: 0.9087, Test Accuracy: 0.6929
Epoch 6, Train Loss: 0.4430, Train Accuracy: 0.9185, Test Accuracy: 0.6421
Epoch 7, Train Loss: 0.3077, Train Accuracy: 0.9359, Test Accuracy: 0.7056
Epoch 8, Train Loss: 0.2336, Train Accuracy: 0.9481, Test Accuracy: 0.7411
Epoch 9, Train Loss: 0.2590, Train Accuracy: 0.9467, Test Accuracy: 0.7157
Epoch 10, Train Loss: 0.1534, Train Accuracy: 0.9610, Test Accuracy: 0.6447
Epoch 11, Train Loss: 0.3007, Train Accuracy: 0.9432, Test Accuracy: 0.6777
Epoch 12, Train Loss: 0.3477, Train Accuracy: 0.9439, Test Accuracy: 0.7234
Epoch 13, Train Loss: 0.1242, Train Accuracy: 0.9735, Test Accuracy: 0.7056
Epoch 14, Train Loss:

In [7]:
import sys
print(sys.executable)

C:\Users\User\anaconda3\python.exe


In [8]:
!{sys.executable} -m pip install tqdm


In [9]:
from tqdm import tqdm
for i in tqdm(range(10)):
    pass



100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


In [10]:
import torch
print(torch.__version__)


2.5.1+cpu
